In [1]:
# Bloco 1: Configuração do Caminho do Dataset

import os

# Caminhos Relativos para o Dataset ISIC
DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")

print("📁 Caminhos do Dataset Configurados:")
print(f"Imagens de Treino: {IMAGE_DIR_TRAIN}")
print(f"Máscaras de Treino: {MASK_DIR_TRAIN}")
print(f"Imagens de Validação: {IMAGE_DIR_VAL}")
print(f"Máscaras de Validação: {MASK_DIR_VAL}")


📁 Caminhos do Dataset Configurados:
Imagens de Treino: ../datasets/image/train
Máscaras de Treino: ../datasets/mask/train
Imagens de Validação: ../datasets/image/validation
Máscaras de Validação: ../datasets/mask/validation


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Bloco Extra: Definição do Modelo Attention U-Net

class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi


class AttentionUNet(nn.Module):
    def __init__(self, num_classes=1):
        super(AttentionUNet, self).__init__()

        # Encoder
        self.encoder1 = self.conv_block(3, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder + Attention
        self.up4 = self.up_conv(1024, 512)
        self.att4 = AttentionBlock(F_g=512, F_l=512, F_int=256)
        self.decoder4 = self.conv_block(1024, 512)

        self.up3 = self.up_conv(512, 256)
        self.att3 = AttentionBlock(F_g=256, F_l=256, F_int=128)
        self.decoder3 = self.conv_block(512, 256)

        self.up2 = self.up_conv(256, 128)
        self.att2 = AttentionBlock(F_g=128, F_l=128, F_int=64)
        self.decoder2 = self.conv_block(256, 128)

        self.up1 = self.up_conv(128, 64)
        self.att1 = AttentionBlock(F_g=64, F_l=64, F_int=32)
        self.decoder1 = self.conv_block(128, 64)

        # Output
        self.output_layer = nn.Conv2d(64, num_classes, kernel_size=1, stride=1, padding=0)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def up_conv(self, in_channels, out_channels):
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, kernel_size=2, stride=2))
        e3 = self.encoder3(F.max_pool2d(e2, kernel_size=2, stride=2))
        e4 = self.encoder4(F.max_pool2d(e3, kernel_size=2, stride=2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, kernel_size=2, stride=2))

        # Decoder + Attention
        d4 = self.up4(b)
        a4 = self.att4(d4, e4)
        d4 = torch.cat((a4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.up3(d4)
        a3 = self.att3(d3, e3)
        d3 = torch.cat((a3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.up2(d3)
        a2 = self.att2(d2, e2)
        d2 = torch.cat((a2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.up1(d2)
        a1 = self.att1(d1, e1)
        d1 = torch.cat((a1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Output
        out = self.output_layer(d1)
        return out


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f4b9a7f5fd0>>
Traceback (most recent call last):
  File "/home/cesar/Pycharm/variacoes_unet/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [3]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definição do dataset customizado
class ISICDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_list = sorted(os.listdir(image_dir))
        self.mask_list = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        # Garante que o índice não vai além do tamanho do dataset
        idx = idx % len(self.image_list)

        # Caminhos das imagens e máscaras
        img_path = os.path.join(self.image_dir, self.image_list[idx])
        mask_name = self.image_list[idx].replace('.jpg', '_segmentation.png')
        mask_path = os.path.join(self.mask_dir, mask_name)

        # Verificação de Existência
        if not os.path.exists(img_path) or not os.path.exists(mask_path):
            print(f"⚠️ Arquivo não encontrado: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        # Carregar Imagem e Máscara
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Verificação de Corrupção
        if image is None or mask is None:
            print(f"⚠️ Imagem ou Máscara corrompida em: {img_path} ou {mask_path}")
            return self.__getitem__((idx + 1) % len(self.image_list))

        # Conversão e Redimensionamento
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        # Normalização
        image = image / 255.0
        mask = (mask > 0).astype(np.float32)  # Converte a máscara para binário (0 ou 1)

        # Conversão para Tensor
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return image, mask

# Diretórios do Dataset
DATASET_DIR = "../datasets"
IMAGE_DIR_TRAIN = os.path.join(DATASET_DIR, "image", "train")
MASK_DIR_TRAIN = os.path.join(DATASET_DIR, "mask", "train")
IMAGE_DIR_VAL = os.path.join(DATASET_DIR, "image", "validation")
MASK_DIR_VAL = os.path.join(DATASET_DIR, "mask", "validation")

# Criar dataset e DataLoader
train_dataset = ISICDataset(IMAGE_DIR_TRAIN, MASK_DIR_TRAIN)
val_dataset = ISICDataset(IMAGE_DIR_VAL, MASK_DIR_VAL)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, drop_last=True)

# Teste: Exibir um batch do DataLoader
batch = next(iter(train_loader))
images, masks = batch

print(f"📊 Formato do batch de imagens: {images.shape}")  # Esperado: [8, 3, 256, 256]
print(f"📊 Formato do batch de máscaras: {masks.shape}")  # Esperado: [8, 1, 256, 256]


📊 Formato do batch de imagens: torch.Size([8, 3, 256, 256])
📊 Formato do batch de máscaras: torch.Size([8, 1, 256, 256])


In [4]:
# Bloco 4: Funções de Perda e Métricas (Mesmo Padrão do U-Net, U-Net++ e ResUNet)

import torch
import torch.nn.functional as F
import torch.optim as optim

# Configuração do dispositivo (usando CPU)
# Configuração do dispositivo (usando CPU)
device = torch.device("cpu")

# Instanciando o modelo Attention U-Net
model = AttentionUNet(num_classes=1)
model.to(device)

# Função de perda (Binary Cross Entropy com Dice Loss)
def dice_loss(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Converte logits para probabilidade
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

criterion = lambda pred, target: 0.5 * F.binary_cross_entropy_with_logits(pred, target) + 0.5 * dice_loss(pred, target)


# Otimizador
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Funções para métricas
def iou(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-5) / (union + 1e-5)

def dice_coefficient(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    return (2. * intersection + 1e-5) / (pred.sum() + target.sum() + 1e-5)

def precision(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    predicted_positive = pred.sum()
    return (true_positive + 1e-5) / (predicted_positive + 1e-5)

def recall(pred, target, threshold=0.5):
    pred = (torch.sigmoid(pred) > threshold).float()
    true_positive = (pred * target).sum()
    actual_positive = target.sum()
    return (true_positive + 1e-5) / (actual_positive + 1e-5)

def f1_score(pred, target, threshold=0.5):
    p = precision(pred, target, threshold)
    r = recall(pred, target, threshold)
    return 2 * (p * r) / (p + r + 1e-5)


In [5]:
# Bloco 5: Função de Treinamento e Validação (Mesmo Padrão do U-Net, U-Net++ e ResUNet)

import torch
import time

# Configuração do dispositivo (usando CPU)
device = torch.device("cpu")
model.to(device)

# Função para treinar o modelo com métricas
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        start_time = time.time()

        # Métricas de treino
        train_iou, train_dice, train_precision, train_recall, train_f1 = 0, 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            # Forward
            outputs = model(images)
            loss = criterion(outputs, masks)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Calcular métricas para o batch
            train_iou += iou(outputs, masks).item()
            train_dice += dice_coefficient(outputs, masks).item()
            train_precision += precision(outputs, masks).item()
            train_recall += recall(outputs, masks).item()
            train_f1 += f1_score(outputs, masks).item()

        # Validação
        model.eval()
        val_loss = 0
        val_iou, val_dice, val_precision, val_recall, val_f1 = 0, 0, 0, 0, 0

        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, masks).item()

                # Calcular métricas para o batch
                val_iou += iou(outputs, masks).item()
                val_dice += dice_coefficient(outputs, masks).item()
                val_precision += precision(outputs, masks).item()
                val_recall += recall(outputs, masks).item()
                val_f1 += f1_score(outputs, masks).item()

        # Exibir progresso
        print(f"📌 Época [{epoch+1}/{epochs}] - Loss Treino: {epoch_loss/len(train_loader):.4f} | Loss Validação: {val_loss/len(val_loader):.4f}")
        print(f"   Métricas de Treino -> IoU: {train_iou/len(train_loader):.4f} | Dice: {train_dice/len(train_loader):.4f} | Prec: {train_precision/len(train_loader):.4f} | Rec: {train_recall/len(train_loader):.4f} | F1: {train_f1/len(train_loader):.4f}")
        print(f"   Métricas de Validação -> IoU: {val_iou/len(val_loader):.4f} | Dice: {val_dice/len(val_loader):.4f} | Prec: {val_precision/len(val_loader):.4f} | Rec: {val_recall/len(val_loader):.4f} | F1: {val_f1/len(val_loader):.4f}")
        print(f"   Tempo da Época: {time.time() - start_time:.2f}s")

# Treinar o modelo com métricas
train_model(model, train_loader, val_loader, epochs=100)


📌 Época [1/100] - Loss Treino: 0.3740 | Loss Validação: 0.3588
   Métricas de Treino -> IoU: 0.6082 | Dice: 0.7472 | Prec: 0.7482 | Rec: 0.7768 | F1: 0.7472
   Métricas de Validação -> IoU: 0.6073 | Dice: 0.7518 | Prec: 0.7188 | Rec: 0.8177 | F1: 0.7517
   Tempo da Época: 1338.37s
📌 Época [2/100] - Loss Treino: 0.2826 | Loss Validação: 0.2759
   Métricas de Treino -> IoU: 0.6856 | Dice: 0.8090 | Prec: 0.8221 | Rec: 0.8223 | F1: 0.8090
   Métricas de Validação -> IoU: 0.6831 | Dice: 0.8085 | Prec: 0.8196 | Rec: 0.8172 | F1: 0.8085
   Tempo da Época: 1334.70s
📌 Época [3/100] - Loss Treino: 0.2455 | Loss Validação: 0.2611
   Métricas de Treino -> IoU: 0.7137 | Dice: 0.8288 | Prec: 0.8424 | Rec: 0.8391 | F1: 0.8288
   Métricas de Validação -> IoU: 0.6940 | Dice: 0.8158 | Prec: 0.8210 | Rec: 0.8272 | F1: 0.8158
   Tempo da Época: 1332.08s
📌 Época [4/100] - Loss Treino: 0.2160 | Loss Validação: 0.2268
   Métricas de Treino -> IoU: 0.7405 | Dice: 0.8473 | Prec: 0.8636 | Rec: 0.8492 | F1: 0.84